# N-Ways to Perform an RF Replay Attack

Hello! In this workshop, you’ll learn how to replay RF signals using SDR! 

Although this project is somewhat basic, it does step-by-step show you the process of transforming radio signals into data bits, recording them and replaying them to do a replay attack. 

We also want you to have fun! Each step includes starter project, and we’ll also provide solutions if you need them. 

Let’s get started!

## Learning Objectives
In this module, you will learn about the following:
- how replay attacks work
- saving waveform to a file
- using the `inspectrum` software for signal analysis
- On-Off-Keying (OOK)
- transmitting using a transmit-capable SDR
- replaying a captured radio signal
- synthesizing signal from bits
- analysis using `rtl-433` and Flipper Zero

## Prerequisites
- general Python knowledge
- general purpose SDR tools or GNU Radio, installed locally.
- RTL-SDR to receive and transmit-capable SDR to transmit RF signals  

# Replay Attacks
Replay attacks are a fundamental technique in wireless security testing, where a previously captured RF signal is retransmitted to trigger the same action—like unlocking a door or turning on a device. In this demonstration, I explore several different tools and workflows that can be used to carry out a replay attack, each with varying levels of complexity and control. From user-friendly devices like the Flipper Zero to more advanced signal processing platforms like GNU Radio, this hands-on approach highlights the strengths and limitations of each method.

There are many ways for replay attacks, and the methods can be chosen based upon your needs and the cost, complexity, and versatility of the devices and tools available.

## Setup
In this module we will be using a simple RF radio device, a **remote control** and **HackRF** in order to receive and transmit signals. This remote is used to turn on and off the lights. Remote has A, B, and OnOff keys on it. We will first supply signal from original transmitter, capture it, and see if we can replay it later. If you have issues capturing a file, or simply you don't have any capturing device with you, you can copy the pre-captured files from the solutions folder into your working directory. There exist `HighTek_A.cf32`, `HighTek_B.cf32`, and `HighTek_OnOff.cf32` pre-captured files in the solutions folder. 

In the sections that follow, we'll first cover what I call a `Simple` replay attack. This does not involve analyzing signal what it contains or what is its modulation scheme. We will simply record and replay. We can do this in several ways. In the first part we will use HackRF along with **GNU Radio**, in the second part **HackRF tool `hackrf_transfer`** will be used. 

Later, in section ... , RF data from the remote control will be analyzed and decoded, bits will be extracted and then we will do replay attack with synthesized signal. This may be somewhat more difficult to do but there are many good open tools for this purpose. What about GNU Radio? Yes, of course GNU Radio is one of that tools, too.   

<img src="nbimages/hightek.png" />

# Simple Replay Attack with GNU Radio
HackRF can be used in conjunction with GNU Radio flow-graphs to capture, save, and replay RF signals. This method requires some prior knowledge on GNU Radio. But the steps we will follow are: 
1. Search the signal of interest  
2. Save the waveform ie, capture
3. Make sure it has been captured correctly
4. Replay the captured signal

## 1.Searching the Signal

The first thing to do is to try to find the exact **operating frequency** of the device. It is a good idea to look at **ISM bands** for this task first. ISM bands or **Industrial Scientific and Medical** bands. You would use such a band is because it's unlicensed in that you are allowed to use equipment that operates in that band without you having a license yourself. There may be rules however about the authorization of the equipment even though there aren't rules about the authorization of the user of the equipment so we call those unlicensed bands. 

<img src="nbimages/ISM_bands.png" alt="Drawing" />

So, we will start by making a flow graph to receive a signal from the remote control and will start with just visualize the signal we have recived. GNU Radio flowgraph sketched below helps you to monitor spectrum for signal of interest. 

<img src="nbimages/grc_1_monitor.png" alt="Drawing" />

## Hint: Running GNU Radio from Terminal
You can run a GNU Radio flow graph right from terminal. 
1. You need to compile the flowgraph, 
2. Then Run the python script generated

Note: Do not forget to specify the filename as parameter if there is. 

There is a pre-made flowgraph, `Monitor.grc` in the solutions directory. 

Click the `...` to run the solution. 

In [ ]:
!grcc solutions/1_Monitor.grc 
!python3 Monitor.py 

**Note 1**: You will see a **continous peak at the center**, i.e., DC spike. It is not a real signal but just an **artifact** from HackRF.  

**Note 2**: You might observe a **mirror image** of the signal if you set gain too high! 

## 2. Saving the Waveform to a File

After adjusting the center frequency for optimum gain and clarity, we can use the following flowgraph `Capture.grc` and then press the peripheral remote button and ultimately capture the RF signal burst to a file. This is just a raw digital waveform. It's the information coming from the Hack RF being saved directly to a file. We are not demodulating, filtering, or doing any kind of processing whatsoever. We are just taking the data from our SDR, Hack RF, and sending it to a file. 

<img src="nbimages/grc_2_capture.png" alt="Drawing" />

There is a pre-made flowgraph, `2_Capture.grc` in the solutions directory. 

Click the `...` to run the solution. 

In [ ]:
!grcc solutions/2_Capture.grc 
!python3 Capture.py --filename='HighTek_A.dat'

## 3. Checking the File
Make sure your record actually contains something meaningful! In order to verify that we have captured our signal of interest we will visualize things. We will again use a QT GUI Frequency Sink however we probably want to slow it down with a throttle block in order to perform in the same speed as real time. We should set the sample rate to be the same sample rate that we have used for the capture which was 2 million samples per second, 2 MSPS. We will give this flow graph a name `CheckRecord.grc`. Run the GR flowgraph below to see what the capture contains. 

<img src="nbimages/grc_3_check.png" alt="Drawing" />

There is a pre-made flowgraph, `3_Check.grc` in the solutions directory. 

Click the `...` to run the solution. 

In [ ]:
!grcc solutions/3_Check.grc 
!python3 Check.py --filename='HighTek_A.dat'

## 4. Replaying a Captured Radio Signal
HackRF can replay (transmit) the raw RF signal in the saved file and thereby invoke the desired peripheral activity without the use of the physical remote. 

<img src="nbimages/grc_4_replay_simple.png" alt="Drawing" />

There is a pre-made flowgraph, `4_Replay.grc` in the solutions directory that simply transmits the recorded signal.  

Click the `...` to run the solution. 

In [ ]:
!grcc solutions/4_Replay.grc 
!python3 Replay.py --filename='HighTek_A.dat'

## 5. Replaying better 
We can do much better if we apply some **gain** and **filtering** before transmission. 

Amplifying our signal digitally allows us to use larger portion of the dynamic range. This is done with the multiplication block. 

After amplification, we apply bandpass filter, being a much better neighbor on the spectrum. 

<img src="nbimages/grc_5_replay_better.png" alt="Drawing" />

Pre-made flowgraph `ReplayBetter.grc` does all these before sending. 

Click the `...` to run the solution. 

In [ ]:
!grcc solutions/5_ReplayBetter.grc 
!python3 ReplayBetter.py --filename='HighTek_A.dat'

# Simple Replay Attack with HackRF Tools
One of the quickest ways to replay an RF signal when the signal center frequency is known is using the HackRF `hackrf_transfer` utility. You can use the following command to capture a radio signal and save it to a file with hackrf_transfer. 

In [ ]:
!hackrf_transfer -s 2000000 -f 314000000 -n 20000000 -r hackrf_transfer_314M_2MHz.cs8 -a 1 -g 32

Having saved the signal, you can transmit it with the following command. 

<mark>Keep in mind that hackrf_transfer utility uses signed 8 bit format by default. So if you just play the GNU Radio record files, it will not work! That's because GNU Radio uses floating point format. </mark>


In [ ]:
!hackrf_transfer -s 2000000 -f 314000000 -R -t hackrf_transfer_314M_2MHz.cs8 -a 1 -x 32

# Simple Replay Attack with Flipper Zero
The Flipper Zero is a portable, open-source multi-tool designed for pentesters, hardware enthusiasts, and hackers. It comes equipped with a range of built-in radios and interfaces, including RFID, infrared, GPIO, iButton, and sub-GHz RF. One of its most popular features is its sub-GHz module, which allows users to interact with a variety of RF-based devices such as garage doors, car key fobs, and remote switches.

To capture and replay an RF signal using Flipper Zero, first open Flipper Lab, web-based platform for the Flipper device. 
<img src="nbimages/Flipper_1_Lab.png" alt="Drawing" />

Navigate to the Sub-GHz menu and select Read mode. 
|**Sub GHz**|**Read Menu**|
|:---:|:---:|
| <img src="nbimages/Flipper_2_Sub_GHz.png" alt="Drawing" /> | <img src="nbimages/Flipper_3_Read.png" alt="Drawing" /> |

Select the configuration like below: 
|**Configuration**|**Scanning...**|
|:---:|:---:|
| <img src="nbimages/Flipper_4_Config.png" alt="Drawing" /> | <img src="nbimages/Flipper_5_Scanning.png" alt="Drawing" /> |

Press the button on the original remote near the Flipper, and it will display the frequency and modulation used. 

Once the signal is captured and saved, you can replay it by selecting the saved file and choosing Send. This feature is particularly useful for security testing of poorly secured or unencrypted RF systems. 
|**Capture**|**Details**|
|:---:|:---:|
| <img src="nbimages/Flipper_6_KeyAPress.png" alt="Drawing" /> | <img src="nbimages/Flipper_7_Details.png" alt="Drawing" /> |

# Decoding with rtl_433
rtl_433 is a versatile open-source tool used with inexpensive USB software-defined radio (SDR) receivers, such as the RTL-SDR dongle. It is designed to decode a wide range of radio transmissions in the 300–928 MHz ISM bands, including signals from weather stations, tire pressure monitors, remote controls, and various IoT sensors. The tool is widely used by hobbyists, researchers, and developers to analyze and collect data from RF devices operating on common unlicensed frequencies.

To capture and analyze a signal from a basic RF remote using rtl_433, connect your RTL-SDR dongle to your machine and launch the rtl_433 command-line tool. When the remote button is pressed near the SDR antenna, rtl_433 attempts to demodulate and decode the signal. If the protocol is recognized, decoded information such as device ID and button press will be printed to the terminal. While RTL_433 is primarily for receiving and decoding signals, not replaying them, the captured data can be saved and further analyzed to understand the protocol. Replay functionality would require a separate SDR transmitter and additional tools like GNU Radio or Universal Radio Hacker (URH).

In [ ]:
!rtl_433 -d "soapy=0,driver=hackrf" -f 314e6 -s 2e6

It should produce an output like below: 
```
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
time      : 2025-08-01 18:38:47
model     : Akhan-100F14 ID (20bit): 0x4b945
Data (4bit): 0x2 (Unlock)
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
```

# Decoding with Universal Readio Hacker (URH)
Universal Radio Hacker (URH) is another powerful open-source application for analyzing unknown wireless protocols. It provides a user-friendly interface for capturing, demodulating, visualizing, and interpreting raw RF signals. URH is commonly used by security researchers and radio enthusiasts to reverse-engineer wireless communication from devices such as remote controls, sensors, and key fobs. It supports a wide range of SDR hardware like HackRF, RTL-SDR, and USRP.

To capture and replay a signal from a basic RF remote using URH, first connect a compatible SDR device (e.g., RTL-SDR or HackRF) and use the Record function in URH to capture the signal while pressing the button on the remote. Once captured, you can analyze the waveform, inspect symbol patterns, and attempt to decode the underlying protocol. URH provides tools to help identify frame structures and bit patterns, even for unknown protocols. After successful analysis, if you're using a device like HackRF that supports transmission, you can use URH’s Transmit feature to replay the captured signal. This makes URH especially useful for reverse-engineering and testing RF devices, but as always, ensure that any capture or replay activities are performed legally and with proper authorization. 

# Decoding with Inspectrum

What we have seen so far is **simple** capture and replay attack. It's pretty good considering how easy it is to accomplish. But keep in mind that we didn't have to decode this signal, we didn't have to even understand what the modulation was. Even if we don't know what the modulation is we could still capture a signal once we've identified it and then replay it and find out if that replay actually does anything. It's a pretty powerful tool to have in our toolbox being able to replay a signal. It's useful for testing systems to see if they respond to a replay packet and it's also extremely useful to create a as a means of creating a repeatable test vector that we can use. 

The cleanest way we could possibly transmit the signal would be actually decoding the information that was transmitted and synthesizing our new waveform that is generated on the CPU without actually capturing any noise at all.  


## A. Decoding with Inspectrum
In order to take a look at what is in this waveform a little bit more in more detail we will use a piece of software called **Inspectrum**.

### 1. Spectrogram View
Inspectrum provides us with spectrogram which is a two-dimensional picture of the information that is saved in the capture
file. What we have is **frequency** on the vertical axis and **time** on the horizontal axis. 

We can open Inspectrum from terminal by specifying sampling rate and file name like below. 

In [ ]:
!inspectrum -r 2e6 HighTek_A.dat

I you scroll right a little bit until, we see things happening a little bit above our center frequency. We can change things like the FFT size which which affects the height of the analysis and we can change the zoom which affects the width. We can also change the color scheme depending on basically setting thresholds of which color are applied to which colors are applied to which power levels. After adjusting things like, you can see that the capture contains 6 bursts of data. The frequency is nearly consistent because everything falls in a horizontal line but the power level goes on and off very dramatically. You can see this is the
beginning of **On-Off-Keying (OOK)** that's pretty cool that we can see that just by looking at the waveform with this tool. 

<img src="nbimages/inspectrum_1_bursts.png" alt="Drawing" />

### 2. Amplitude Plot
Let's take a closer look at one of the bursts. Zooming in on time axis, we can see clearly that  it transmits for a while and then it's just back to noise and then it transmits for a while and then we're back to noise again. Let's take a look at how we can demodulate this and the nice thing is that Spectrum actually has some built-in demodulation capability. You have to kind of do a lot of things by hand but you can demodulate a signal like this using nothing but in Spectrum. So, we know that the amplitude is being modulated here the transmitter is basically being switched from full amplitude to zero amplitude that is completely off so what we need to do is add an **amplitude plot**. The the red line in the center is the center frequency of my tuning and the white lines at the top and bottom are showing the edges of the filter that I'm using to filter this channel and so you can kind of adjust if you want a wider filter you can drag them out and if you want a narrower filter. 

<img src="nbimages/inspectrum_2_amplitude_plot.png" alt="Drawing" />

### 3. Thresholding
The next thing that we can do is you can take this amplitude demodulated signal and add another derived plot, **threshold** plot, and now anything below this dotted line will be counted as a zero and anything above the dotted line will be counted as a one. So, now we have a nice clean representation of kind of all the ons and offs in this signal. 

<img src="nbimages/inspectrum_3_thresholding.png" alt="Drawing" /> 

### 4. Measuring the baudrate
The next thing we need to do is figure out when we're going to measure this signal to see you know where are the ones and zeros and to do that in Spectrum you need to turn on the **cursors**. Cursors have a bunch of uses you can also use them just to measure the baud rates of signals. 

<img src="nbimages/inspectrum_4_measuring_baud.png" alt="Drawing" /> 

### 5. Obtaining Bits
We can start by using cursor for one bit period but eventually once you have the cursor set up properly you can also use them to demodulate and extract data from things. Just looking at kind of one bit period where the transmitter is on you can get a rough idea how long burst will be. Then gradually increase symbol count to 10, 50, and finally to 100. Make sure that we're staying lined up with the bit periods there like my dotted lines are lining up exactly with the bit transitions. So, once you have everything lined up and we're lined up on the threshold plot you can right click there and say **extract symbols**. You can either go to standard output or to the  clipboard. If you choose clipboard, clipboard should have a set of bits okay one zero one zero one zero that sounds about right for the start of the signal and then eventually we get into where the actual bits of the message are. 

<img src="nbimages/inspectrum_5_bits.png" alt="Drawing" /> 

In [ ]:
import bitarray
# read data from file
openFile = open("solutions/bits_A.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

In [ ]:
import bitarray
# read data from file
openFile = open("solutions/bits_B.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

In [ ]:
import bitarray
# read data from file
openFile = open("solutions/bits_OnOff.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

# Replay Attack by Synthesizing the Signal with GNU Radio
The next step is to create the desired signal on the fly, directly from GNURadio. Below is the flowgraph that can be used to tranmit the bits we have obtained preivously over the air.  

<img src="nbimages/grc_6_synthesize.png" alt="Drawing" /> 